<a href="https://colab.research.google.com/github/Sparrow0hawk/crime_sim_toolkit/blob/master/Forces_%2B_LSOAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate a reference document matching LSOAs to Police Force codes

Some data is retrieved by link shown in the code.

Other data was downloaded from data.police for all forces for 2019-06 and uploaded into the colabs environment.

## Revision 14/08/2019

Issues arose from using the Falls within category from data.police.

Therefore matching is now performed using Community Safety Partnership (analogous to local authority) using an established mapping dataset.

In [0]:
import pandas as pd
import glob
import numpy as np

In [2]:
# get sheet with LSOA/MSOA/LA breakdown
! wget -O table_2011_MSOA_LSOA.zip https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales/rft-table-php01-2011-msoas-and-lsoas.zip

--2019-08-14 10:56:02--  https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/2011censuspopulationandhouseholdestimatesforwardsandoutputareasinenglandandwales/rft-table-php01-2011-msoas-and-lsoas.zip
Resolving www.ons.gov.uk (www.ons.gov.uk)... 104.20.60.76, 104.20.61.76, 2606:4700:10::6814:3c4c, ...
Connecting to www.ons.gov.uk (www.ons.gov.uk)|104.20.60.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘table_2011_MSOA_LSOA.zip’

table_2011_MSOA_LSO     [ <=>                ]   2.87M  --.-KB/s    in 0.08s   

2019-08-14 10:56:03 (34.5 MB/s) - ‘table_2011_MSOA_LSOA.zip’ saved [3009622]



In [3]:
# get CSP data with matching
! wget -O recordedcrimeatcsplevelyearendingmarch2019.xls https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/crimeandjustice/datasets/recordedcrimedatabycommunitysafetypartnershiparea/current/recordedcrimeatcsplevelyearendingmarch2019.xls

--2019-08-14 10:56:04--  https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/crimeandjustice/datasets/recordedcrimedatabycommunitysafetypartnershiparea/current/recordedcrimeatcsplevelyearendingmarch2019.xls
Resolving www.ons.gov.uk (www.ons.gov.uk)... 104.20.60.76, 104.20.61.76, 2606:4700:10::6814:3c4c, ...
Connecting to www.ons.gov.uk (www.ons.gov.uk)|104.20.60.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.ms-excel]
Saving to: ‘recordedcrimeatcsplevelyearendingmarch2019.xls’

recordedcrimeatcspl     [  <=>               ] 860.50K  2.68MB/s    in 0.3s    

2019-08-14 10:56:05 (2.68 MB/s) - ‘recordedcrimeatcsplevelyearendingmarch2019.xls’ saved [881152]



In [4]:
# unzip file
! unzip table_2011_MSOA_LSOA.zip

Archive:  table_2011_MSOA_LSOA.zip
  inflating: R1_4_EW__RT__Table_PHP01___LSOA_MSOA_v4.xls  


In [0]:
PRCLE_data = pd.read_excel('/content/recordedcrimeatcsplevelyearendingmarch2019.xls', sheet_name='Table C2', skiprows=4)

In [0]:
PRCLE_data = PRCLE_data.iloc[:,:6]

PRCLE_data.dropna(how='any',inplace=True)

In [10]:
PRCLE_data.head()

,Police Force Area code,Police Force Area name,Community Safety Partnership code,Community Safety Partnership name,Local Authority code,Local Authority name
2,E23000036,Avon and Somerset,E22000001,Bath and North East Somerset,E06000022,Bath and North East Somerset
3,E23000036,Avon and Somerset,E22000002,"Bristol, City of",E06000023,"Bristol, City of"
4,E23000036,Avon and Somerset,E22000003,North Somerset,E06000024,North Somerset
6,E23000036,Avon and Somerset,E22000006,South Gloucestershire,E06000025,South Gloucestershire
9,E23000026,Bedfordshire,E22000009,Bedford,E06000055,Bedford


In [0]:
# open file on sheet five skipping useless rows
ONS_geo_ref = pd.read_excel('R1_4_EW__RT__Table_PHP01___LSOA_MSOA_v4.xls', sheet_name=5, skiprows=11)

In [13]:
ONS_geo_ref.columns

Index(['Region code', 'Region name', 'Local authority code',
       'Local authority name', 'MSOA Code', 'MSOA Name', 'LSOA Code',
       'LSOA Name', 'Unnamed: 8', 'Persons', 'Persons.1', 'Persons.2',
       'Unnamed: 12', 'Hectares', 'Persons per hectare', 'Unnamed: 15',
       'Households', 'Persons per household'],
      dtype='object')

In [0]:
# select out desired columns
ONS_geo_ref1 = ONS_geo_ref[['Region code', 'Region name', 'Local authority code',
       'Local authority name', 'MSOA Code', 'MSOA Name', 'LSOA Code',
       'LSOA Name']]

In [22]:
# drop empty rows
ONS_geo_ref1.dropna(how='any', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
ONS_geo_ref1.head()

,Region code,Region name,Local authority code,Local authority name,MSOA Code,MSOA Name,LSOA Code,LSOA Name
3,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020634,County Durham 001A
4,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020635,County Durham 001B
5,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020636,County Durham 001C
6,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020654,County Durham 001D
7,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020676,County Durham 001E


In [18]:
PRCLE_data[PRCLE_data['Local Authority code'].isin(['E06000047'])]

,Police Force Area code,Police Force Area name,Community Safety Partnership code,Community Safety Partnership name,Local Authority code,Local Authority name
62,E23000008,Durham,E22000365,County Durham,E06000047,County Durham


In [24]:
# build a comprehensive matching table between LSOA/MSOA and police force using local authority
# using apply to match LA code and then add police force

ONS_geo_ref1['Police_force'] = ONS_geo_ref1['Local authority code'].map(lambda x: PRCLE_data[PRCLE_data['Local Authority code'].isin([x])]['Police Force Area name'].tolist()[0] if len(PRCLE_data[PRCLE_data['Local Authority code'].isin([x])]['Police Force Area name'].tolist()) > 0 else np.NaN)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
# add this column so we now have LSOA code, local authority name and police force 
ONS_geo_ref1.head()

,Region code,Region name,Local authority code,Local authority name,MSOA Code,MSOA Name,LSOA Code,LSOA Name,Police_force
3,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020634,County Durham 001A,Durham
4,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020635,County Durham 001B,Durham
5,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020636,County Durham 001C,Durham
6,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020654,County Durham 001D,Durham
7,E12000001,NORTH EAST,E06000047,County Durham UA,E02004297,County Durham 001,E01020676,County Durham 001E,Durham


In [0]:
# now happy to proceed with saving this list to use as reference list between LSOA and police force
ONS_geo_ref1.to_csv('PoliceforceLSOA.csv')

In [0]:
# prompt download from colab
from google.colab import files

files.download('PoliceforceLSOA.csv')